In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import col

# Set the location of the Delta Lake and Kafka packages
delta_package = "io.delta:delta-spark_2.12:3.0.0"  # Replace with the correct Delta version
kafka_package = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1"  # Replace with the correct Spark version
xml_package = "com.databricks:spark-xml_2.12:0.14.0"

# Initialize Spark Session with Delta Lake and Kafka support
spark = SparkSession.builder \
    .appName("test") \
    .config("spark.jars.packages", f"{delta_package},{xml_package}") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/spark/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e0a662cf-5d9d-4117-bb67-be524cca904a;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 260ms :: artifacts dl 7ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [defa

In [2]:
raw_df = spark.read.format('parquet').option("mergeSchema", "true").load('hdfs://spark-test1:9000/raw/transactions')
raw_df_20 = raw_df.orderBy(col("_raw_insert_timestamp").desc()).limit(20)
print(raw_df.count())

953


In [3]:
processed_df = spark.read.format('delta').option("mergeSchema", "true").load('hdfs://spark-test1:9000/processed/transactions')
processed_df_20 = processed_df.orderBy(col("_processed_insert_timestamp").desc()).limit(20)
display(processed_df_20.toPandas())

23/12/12 21:32:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,TransactionId,Amount,CustomerId,TransactionDateTime,TransactionDate,Location,Result,_processed_insert_timestamp,_raw_insert_timestamp
0,6,236.880005,76,2023-12-12 20:47:44,2023-12-12,LONDON,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:49.281194
1,84,87.599998,30,2023-12-12 20:47:21,2023-12-12,SYDNEY,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:25.595623
2,51,193.429993,59,2023-12-12 20:46:47,2023-12-12,NEW YORK,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:00.183180
3,74,320.899994,80,2023-12-12 20:47:30,2023-12-12,SYDNEY,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:35.376694
4,93,166.500000,99,2023-12-12 20:47:42,2023-12-12,NEW YORK,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:46.239144
5,35,793.010010,99,2023-12-12 20:47:21,2023-12-12,TOKYO,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:25.595623
6,56,378.929993,87,2023-12-12 20:47:24,2023-12-12,TOKYO,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:27.955291
7,61,404.859985,6,2023-12-12 20:47:38,2023-12-12,TOKYO,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:42.101083
8,15,960.969971,90,2023-12-12 20:47:24,2023-12-12,SYDNEY,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:27.955291
9,87,30.610001,16,2023-12-12 20:47:38,2023-12-12,TOKYO,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:42.101083
